In [5]:
import json
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

In [16]:
image_path = '/Users/gauravneupane/Documents/ml/data/datasets/train/image'
annotation_path = '/Users/gauravneupane/Documents/ml/data/datasets/train/annos'

In [7]:
#'trousers': 8,
# 'short sleeve top': 1,
# 'long sleeve dress': 11,
#'long sleeve top': 2,
#'skirt': 9,
#'shorts': 7,
#'long sleeve outwear': 4,
#'vest dress': 12,
#'short sleeve dress': 10,
#'vest': 5,
#'sling dress': 13,
#'short sleeve outwear': 3,
#'sling': 6

color_map = {
    0: [0, 0, 0],        # Background: Black
    1: [0, 128, 0],     
    2: [255, 0, 0], 
    3: [0, 0, 255],  
    4: [0, 255, 0],     
    5: [255, 255, 0], 
    6: [255, 165, 0], 
    7: [255, 192, 203],
    8: [128, 0, 128], 
    9: [0, 255, 255], 
    10: [128, 128, 0], 
    11: [255, 0, 255], 
    12: [0, 128, 128], 
    13: [128, 128, 128], 
    14: [0, 255, 255],   
}


In [8]:
def apply_color_map(mask, color_map):
    # Convert mask to RGB image using the color map
    rgb_mask = np.zeros((mask.shape[0], mask.shape[1], 3), dtype=np.uint8)
    for label, color in color_map.items():
        rgb_mask[mask == label] = color
    return rgb_mask

In [9]:
def overlay_mask_on_image(image, mask, color_map):
    # Convert mask to RGB format
    mask_rgb = apply_color_map(mask, color_map)
    
    # Overlay mask on the image
    overlay = cv2.addWeighted(image, 0.7, mask_rgb, 0.3, 0)
    return overlay

In [10]:
def load_json(json_path):
    with open(json_path, 'r') as file:
        data = json.load(file)
    return data

In [11]:
exclude_keys = ['source','pair_id']
def create_mask_for_whole_image(image_shape, segmentation_cats):
    '''create new mask image from the segmentation points'''
    mask = np.zeros((image_shape[0], image_shape[1]), dtype=np.int8)
    for data in segmentation_cats:
        segmentation = data['data_points']
        class_label = data['class_label']
        for seg in segmentation:
            points = np.array(seg).reshape(-1, 2)
            cv2.fillPoly(mask, [points.astype(np.int32)], class_label)
    return mask

In [61]:
def build_mask(image_shape,json_data):
    '''extract mask information as points from the dataset and create a new image as a mask'''
    image_segmentation_data_points = []
    filtered_keys = [key for key in json_data.keys() if key not in exclude_keys]
    categories = []
    for key in filtered_keys:
        data_points = json_data[key]['segmentation']
        category = json_data[key]['category_name']
        class_label = json_data[key]['category_id']
        segmentation_with_class = {'data_points': data_points,
                                   'category': category,
                                   'class_label': class_label}
        image_segmentation_data_points.append(segmentation_with_class)
        categories.append(category)
    mask = create_mask_for_whole_image(image_shape,image_segmentation_data_points)
    return mask, categories

In [83]:
images_per_class = 1000
image_annot_save_path = '/Users/gauravneupane/Documents/ml/projects/image_retrieval/new'
output_image_shape = (256,256)
category_count = {'trousers': 0,
                    'short sleeve top': 0,
                    'long sleeve dress': 0,
                    'long sleeve top': 0,
                    'skirt': 0,
                    'shorts': 0,
                    'long sleeve outwear': 0,
                    'vest dress': 0,
                    'short sleeve dress': 0,
                    'vest': 0,
                    'sling dress': 0,
                    'short sleeve outwear': 0,
                    'sling': 0
                    }
loop_count = 0
for file in sorted(os.listdir(image_path)):
    stop = True
    full_path = os.path.join(image_path, file)
    annot_path = os.path.join(annotation_path, file.split('.')[0]+'.json')
    annot_data = load_json(annot_path)
    image = cv2.imread(full_path)
    # image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    mask,categories = build_mask(image.shape,annot_data)
    category_full = True
    for category in categories:
        if(category_count[category] < images_per_class):
            category_count[category] = category_count[category] + 1
            category_full = False
    for value in category_count.values():
        if(value < images_per_class):
            stop = False
    if(stop):
        break
    if not category_full:
        print(category_count)
    loop_count += 1
    resized_image = cv2.resize(image, output_image_shape, interpolation = cv2.INTER_NEAREST)
    # normalizing image and changing type to float32 which is essential for deep learning models
    normalized_image = cv2.normalize(resized_image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    resized_mask = cv2.resize(mask, output_image_shape, interpolation = cv2.INTER_NEAREST)
    image_save_path = os.path.join(image_annot_save_path, file)
    annot_save_path = os.path.join(image_annot_save_path, 'annot-'+file)
    cv2.imwrite(image_save_path, resized_image)
    cv2.imwrite(annot_save_path, resized_mask)
        # # overlay = overlay_mask_on_image(image_rgb, mask, color_map)
        # # plt.imshow(overlay)

{'trousers': 1, 'short sleeve top': 1, 'long sleeve dress': 0, 'long sleeve top': 0, 'skirt': 0, 'shorts': 0, 'long sleeve outwear': 0, 'vest dress': 0, 'short sleeve dress': 0, 'vest': 0, 'sling dress': 0, 'short sleeve outwear': 0, 'sling': 0}
{'trousers': 1, 'short sleeve top': 3, 'long sleeve dress': 0, 'long sleeve top': 0, 'skirt': 0, 'shorts': 0, 'long sleeve outwear': 0, 'vest dress': 0, 'short sleeve dress': 0, 'vest': 0, 'sling dress': 0, 'short sleeve outwear': 0, 'sling': 0}
{'trousers': 1, 'short sleeve top': 3, 'long sleeve dress': 1, 'long sleeve top': 0, 'skirt': 0, 'shorts': 0, 'long sleeve outwear': 0, 'vest dress': 0, 'short sleeve dress': 0, 'vest': 0, 'sling dress': 0, 'short sleeve outwear': 0, 'sling': 0}
{'trousers': 1, 'short sleeve top': 3, 'long sleeve dress': 2, 'long sleeve top': 0, 'skirt': 0, 'shorts': 0, 'long sleeve outwear': 0, 'vest dress': 0, 'short sleeve dress': 0, 'vest': 0, 'sling dress': 0, 'short sleeve outwear': 0, 'sling': 0}
{'trousers': 1, 

KeyboardInterrupt: 

In [ ]:
{'trousers': 8, 'short sleeve top': 1, 'long sleeve dress': 11, 'long sleeve top': 2, 'skirt': 9, 'shorts': 7, 'long sleeve outwear': 4, 'vest dress': 12, 'short sleeve dress': 10, 'vest': 5, 'sling dress': 13, 'short sleeve outwear': 3, 'sling': 6}